In [6]:
import os
from typing import List, Dict, Any
import pandas as pd


In [7]:
from langchain_core.documents import Document
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)
print("All imports successful")

All imports successful


In [8]:
doc = Document(
    page_content="This is the main text content that will be embedded and searched.", 
    metadata={""
    "source": "example.txt",
    "page": 1,
    "author": "Krish Naik",
    "date_created": "2024-01-01",
    "custom_field": "any_value"
    }
)
print(f"Content: {doc.page_content}")
print(f"Metadata: {doc.metadata}")

Content: This is the main text content that will be embedded and searched.
Metadata: {'source': 'example.txt', 'page': 1, 'author': 'Krish Naik', 'date_created': '2024-01-01', 'custom_field': 'any_value'}


In [9]:
import os
os.makedirs("data/text_files", exist_ok=True)

In [10]:
sample_text = {
    "data/text_files/python_intro.txt": """
    This is the content of file one. It contains important information about LangChain. 
    LangChain is a powerful library for building applications with language models.
    
    
    It provides tools for data ingestion, parsing, and embedding.
    and more.
    
    Key features include document loaders, text splitters, and vector stores.
    Python is a versatile programming language used for various applications, including web development, data analysis, and artificial intelligence.


    Learning Python can open up many career opportunities in the tech industry.
    """,
    
}

for filepath, content in sample_text.items():
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)

print("Sample text files created successfully.")        

Sample text files created successfully.


## File Loader

In [13]:
from langchain_community.document_loaders import TextLoader

## Loading single text file
loader = TextLoader("data/text_files/python_intro.txt", encoding="utf-8")

documents = loader.load()
print(type(documents))
print(documents[0].metadata)


<class 'list'>
{'source': 'data/text_files/python_intro.txt'}


## Directory loader

In [20]:
from langchain_community.document_loaders import DirectoryLoader

# Loading all text files from a directory
directory_loader = DirectoryLoader(
    "data/text_files", 
    glob="**/*.txt", ## pattern to match files
    loader_cls=TextLoader, ## loader class to use
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True
)

documents = directory_loader.load()
for i, doc in enumerate(documents):
    print(f"\nDocument {i+1}:")
    print(f"  Source: {doc.metadata['source']}")
    print(f"  Length of content: {len(doc.page_content)} characters")


100%|██████████| 1/1 [00:00<00:00, 1501.72it/s]


Document 1:
  Source: data/text_files/python_intro.txt
  Length of content: 570 characters


## Test splittings strategies

In [23]:
from langchain_text_splitters import ( 
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)
print(documents)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='\n    This is the content of file one. It contains important information about LangChain. \n    LangChain is a powerful library for building applications with language models.\n\n\n    It provides tools for data ingestion, parsing, and embedding.\n    and more.\n\n    Key features include document loaders, text splitters, and vector stores.\n    Python is a versatile programming language used for various applications, including web development, data analysis, and artificial intelligence.\n\n\n    Learning Python can open up many career opportunities in the tech industry.\n    ')]


In [24]:
### Method 1 - CHaracter Text Splitter
text=documents[0].page_content
text

'\n    This is the content of file one. It contains important information about LangChain. \n    LangChain is a powerful library for building applications with language models.\n\n\n    It provides tools for data ingestion, parsing, and embedding.\n    and more.\n\n    Key features include document loaders, text splitters, and vector stores.\n    Python is a versatile programming language used for various applications, including web development, data analysis, and artificial intelligence.\n\n\n    Learning Python can open up many career opportunities in the tech industry.\n    '

In [ ]:
print("1 CHARACTER TEXT SPLITTER")
char_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=200, 
    chunk_overlap=20,
    length_function=len
)

char_chunks=char_splitter.split_text(text)
print(f"Number of chunks: {len(char_chunks)}")
print(f"First chunk:\n{char_chunks[0][:100]}...")

1 CHARACTER TEXT SPLITTER
Number of chunks: 4
First chunk:
This is the content of file one. It contains important information about LangChain. 
    LangChain i...
This is the content of file one. It contains important information about LangChain. 
    LangChain is a powerful library for building applications with language models.


In [28]:
print(char_chunks[0])
print(char_chunks[1])


This is the content of file one. It contains important information about LangChain. 
    LangChain is a powerful library for building applications with language models.
It provides tools for data ingestion, parsing, and embedding.
    and more.
    Key features include document loaders, text splitters, and vector stores.


In [29]:
print("2 Recursive CHARACTER TEXT SPLITTER")
recursive_char_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=200, 
    chunk_overlap=20,
    length_function=len
)

recursive_chunks = recursive_char_splitter.split_text(text)
print(f"Number of chunks: {len(recursive_chunks)}")
print(f"First chunk:\n{recursive_chunks[0][:100]}...")


2 Recursive CHARACTER TEXT SPLITTER
Number of chunks: 5
First chunk:
This is the content of file one. It contains important information about LangChain. 
    LangChain i...


In [ ]:
print(recursive_chunks[0])
print("SPLITTER")
print(recursive_chunks[1])
print("SPLITTER")
print(recursive_chunks[2])
print("SPLITTER")
print(recursive_chunks[3])
print("SPLITTER")
print(recursive_chunks[4])


This is the content of file one. It contains important information about LangChain. 
    LangChain is a powerful library for building applications with language models.
SPLITTER
It provides tools for data ingestion, parsing, and embedding.
    and more.
SPLITTER
Key features include document loaders, text splitters, and vector stores.
SPLITTER
Python is a versatile programming language used for various applications, including web development, data analysis, and artificial intelligence.
SPLITTER
Learning Python can open up many career opportunities in the tech industry.


In [ ]:
# Method3: Token based splitter
token_splitter = TokenTextSplitter(
    chunk_size=50, #calculate spaces and not only characters
    chunk_overlap=20
)

token_chunks = token_splitter.split_text(text)
print(f"Number of chunks: {len(token_chunks)}")
print(f"First chunk:\n{token_chunks[0][:100]}...")


Number of chunks: 4
First chunk:

    This is the content of file one. It contains important information about LangChain. 
    LangCh...


In [ ]:
# Comparison
print("\n📊 Text Splitting Methods Comparison:")
print("\nCharacterTextSplitter:")
print("  ✅ Simple and predictable")
print("  ✅ Good for structured text")
print("  ❌ May break mid-sentence")
print("  Use when: Text has clear delimiters")

print("\nRecursiveCharacterTextSplitter:")
print("  ✅ Respects text structure")
print("  ✅ Tries multiple separators")
print("  ✅ Best general-purpose splitter")
print("  ❌ Slightly more complex")
print("  Use when: Default choice for most texts")

print("\nTokenTextSplitter:")
print("  ✅ Respects model token limits")
print("  ✅ More accurate for embeddings")
print("  ❌ Slower than character-based")
print("  Use when: Working with token-limited models")